In [1]:
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.tracker import Tracker as Deeptrack
from deep_sort.tools import generate_detections as gdet

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.13.0'

In [4]:

pwd

'/Users/aayush/speed/env/project_speed'

In [5]:
class Tracker:
    tracker = None
    encoder = None
    tracks = None

    def __init__(self):
        max_cosine_distance = 0.4
        nn_budget = None

        encoder_model_filename = 'resources/networks/mars-small128.pb'

        metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
        self.tracker = Deeptrack(metric)
        self.encoder = gdet.create_box_encoder(encoder_model_filename, batch_size=1)

       

    def update(self, frame, detections):

        if len(detections) == 0:
            self.tracker.predict()
            self.tracker.update([])  
   
            self.update_tracks()
            return

 
        bboxes = np.asarray([d[:-1] for d in detections])
        
        bboxes[:, 2:] = bboxes[:, 2:] - bboxes[:, 0:2]
  
        scores = [d[-1] for d in detections]

     
        features = self.encoder(frame, bboxes)

        dets = []
        for bbox_id, bbox in enumerate(bboxes):
            dets.append(Detection(bbox, scores[bbox_id], features[bbox_id]))

        self.tracker.predict()
        self.tracker.update(dets)
        self.update_tracks()

    def update_tracks(self):
        tracks = []
        for track in self.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()

            id = track.track_id
    
            
            tracks.append(Track(id, bbox))

        self.tracks = tracks


class Track:
    track_id = None
    bbox = None

    def __init__(self, id, bbox):
        self.track_id = id
        self.bbox = bbox

In [6]:
import math
def estimate_speed(time1,time2 ):
    #Euclidean Distance Formula
    
    dist = 300
    speed = dist/(time2 - time1)
    return speed*3.6

In [8]:
from ultralytics import YOLO
import matplotlib
import matplotlib.pyplot as plt
import easyocr
def liscence_plate(name_of_img , bbox , track_id):
    x1, y1, x2, y2 = bbox
    previous_directory = os.getcwd()
    os.chdir('./number_plate_detection_img/frame')
   

    img =  plt.imread(name_of_img )
    os.chdir(previous_directory)
    plt.imshow(img)
  
    
    crop_to_car = img[int(y1):int(y2) , int(x1):int(x2)  ]
    
    os.chdir('./number_plate_detection_img/bbox')
    plt.imsave(f'imgcar{track_id}.png' , crop_to_car)
    img = cv2.imread(f'imgcar{track_id}.png')
    plt.imshow(crop_to_car)
    os.chdir(previous_directory)
    license_plate_detector = YOLO('./license_plate/license_plate_detector.pt')
    
   
    detect = license_plate_detector(img)

    
    for detections_ in detect :
        detection = []
        if detections_!=[]:
            
            for det in detections_.boxes.data.tolist():
                xmin  = int(det[0])
                ymin = int(det[1])
                xmax = int(det[2])
                ymax = int(det[3])
                confidence = det[4]
               
                detection.append([xmin , ymin , xmax , ymax , confidence ])

    
        
    if detection!=[]:
        crop = img[ymin : ymax , xmin:xmax , :]
        license_plate_crop_gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
        _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)
        os.chdir('./license_plate')
        plt.imsave(f'lisc{track_id}.png' , license_plate_crop_thresh)
        
     
        os.chdir(previous_directory)
        
        reader = easyocr.Reader(['en'])
        result = reader.readtext(license_plate_crop_thresh)
        print(result)
        if result!=[] :
            if result[0][-1]>0.5 :
                
                return result[0][-2]
            else:
                return "Not Accurate yet"
        else:
            return None
    else:
        return None


In [9]:

import os
previous_directory = os.getcwd()
os.getcwd()

'/Users/aayush/speed/env/project_speed'

In [10]:
#os.chdir('./number_plate_detection_img/bbox')

# img = cv2.imread('imgcar1.png')
# plt.imshow(img)
# os.chdir(previous_directory)

In [11]:
pwd

'/Users/aayush/speed/env/project_speed'

In [ ]:
import numpy as np
import torch
import os
import cv2
import pandas
from ultralytics import YOLO
import random
import time
import matplotlib
import matplotlib.pyplot as plt
model = YOLO('yolov8m.pt')
cap = cv2.VideoCapture(os.path.join( 'hey.mp4'))

tracker = Tracker()

threshold  = 0.5
# colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]
ret , frame = cap.read()

video_out_path = os.path.join('.', 'ijuii.mp4')


vehicle_entering_the_area = {}

vehicle_enter_list = []

time_enter = {}

time_exit = {}

total_frames = {}

speed = {}

speed_exceed = {}

number_plate = {}

not_acc = {}

count = 0
while ret:
   
    
    # Make detections 
    results = model(frame)

    area = [(1383,834) , (1206,1020) , (3183,1020) ,(2928,834) ]
    area2 = [(762,1698) , (492,2157) , (3834,2157) , (3834,1698)]
    for result in results : 
        detections = []
        for det in result.boxes.data.tolist() :
            xmin  = int(det[0])
            ymin = int(det[1])
            xmax = int(det[2])
            ymax = int(det[3])
            confidence = det[4]
         
            if confidence > threshold :
                detections.append([xmin , ymin , xmax , ymax , confidence ])
        
        tracker.update(frame , detections)
        
        for tracks in tracker.tracks: 
            bbox = tracks.bbox
            x1, y1, x2, y2 = bbox
            track_id = tracks.track_id
            cx = int((x1+x2)/2)
            cy = int((y1+y2)/2)
            if cv2.pointPolygonTest(np.array(area , np.int32), (cx,cy) , False) > 0 :
                
                time_enter[track_id] = time.time()
                vehicle_entering_the_area[track_id] = (cx,cy)
                
            if cv2.pointPolygonTest(np.array(area2 , np.int32), (cx,cy) , False) > 0 and track_id in vehicle_entering_the_area : #for the time changing this code to check speed in uper region
                time_exit[track_id] = time.time()
                frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), [0,0,0], 5)
                cv2.putText(frame, f'id : {track_id}', (int(x1), int(y1-10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (36,255,12), 2)
                cv2.circle(frame , (cx,cy) , 5 , [255,255,0] , -1)
                if track_id not in speed :
                    speed_vehicle = estimate_speed(time_enter[track_id] ,time_exit[track_id])
                    speed[track_id] = speed_vehicle
                cv2.putText(frame, f'speed : {speed[track_id]}', (int(x1), int(y1+10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (36,255,12), 2)
                if speed[track_id] > 8 :
                    
                    speed_exceed[track_id] = speed
                    previous_directory = os.getcwd()
                    os.chdir('./number_plate_detection_img/frame')
                    plt.imsave(f'img{track_id}.png' , frame)
                    os.chdir(previous_directory)
                    text = liscence_plate(f'img{track_id}.png' , bbox , track_id)
                   
                    not_acc[track_id] = text
                    if text != 'Not Accurate yet' or None : 
                        
                        number_plate[track_id] = text
                else :
                    cv2.putText(frame, 'Speed Ok!!', (int(x1), int(y1+40)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                
                    
                    
                    
                    
            elif track_id in vehicle_entering_the_area :
                
                frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), [0,0,0], 5)
                cv2.putText(frame, f'id : {track_id}', (int(x1), int(y1-10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (36,255,12), 2)
                cv2.circle(frame , (cx,cy) , 5 , [255,255,0] , -1)
    

            if track_id in not_acc  :
                if track_id in number_plate :
                
                    cv2.putText(frame, f'number_plate : {number_plate[track_id]}', (int(x1), int(y1+40)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                else:

                    cv2.putText(frame, 'Not Accurate yet', (int(x1), int(y1+40)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

           
                
                
    cv2.polylines(frame , [np.array(area , np.int32)] ,True , (15,220,10) , 6 )
    cv2.polylines(frame , [np.array(area2 , np.int32)] ,True , (15,220,10) , 6 )

    cv2.imshow('yolo' , frame)
    ret, frame = cap.read()
    count =0
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


0: 384x640 1 person, 22 cars, 1 bus, 5 trucks, 125.3ms
Speed: 4.8ms preprocess, 125.3ms inference, 7.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22 cars, 1 bus, 5 trucks, 115.6ms
Speed: 1.8ms preprocess, 115.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 5 trucks, 112.0ms
Speed: 2.0ms preprocess, 112.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 21 cars, 1 bus, 4 trucks, 105.3ms
Speed: 1.5ms preprocess, 105.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 20 cars, 1 bus, 5 trucks, 114.8ms
Speed: 1.8ms preprocess, 114.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 5 trucks, 112.5ms
Speed: 1.5ms preprocess, 112.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 4 trucks, 111.7ms
Speed: 1.6ms preprocess, 111.7ms inference, 0.7ms 